In [2]:
import os
import io
import IPython.display
from PIL import Image
import base64
from dotenv import load_dotenv, find_dotenv
import requests
import json
import gradio as gr

# Load environment variables from .env file
_ = load_dotenv(find_dotenv())

# Access environment variables
hf_api_key = "your_huggingface_api_key_here"
ENDPOINT_URL = "https://api-inference.huggingface.co/models/your-model-id"
PORT = int(os.environ.get('PORT1', 7860))  # Default to 7860 if PORT1 not set

# Function to call Hugging Face API for image captioning
def get_completion(inputs, parameters=None):
    headers = {
        "Authorization": f"Bearer {hf_api_key}",
        "Content-Type": "application/json",
    }
    data = {"inputs": inputs}
    if parameters:
        data["parameters"] = parameters
    response = requests.post(ENDPOINT_URL, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError(f"Error {response.status_code}: {response.text}")

# Convert PIL image to base64 string
def image_to_base64_str(pil_image):
    byte_arr = io.BytesIO()
    pil_image.save(byte_arr, format="PNG")
    byte_arr = byte_arr.getvalue()
    return base64.b64encode(byte_arr).decode("utf-8")

# Generate wildlife caption from image
def wildlife_captioner(image):
    base64_image = image_to_base64_str(image)
    result = get_completion(base64_image)
    return result[0].get("generated_text", "No caption generated.")

# Display sample wildlife image (optional)
sample_image_url = "https://images.unsplash.com/photo-1506744038136-46273834b3fb"  # Example wildlife image
IPython.display.display(IPython.display.Image(url=sample_image_url))

# Create Gradio interface for wildlife captions
gr.close_all()  # Ensure no previous Gradio instances are running

demo = gr.Interface(
    fn=wildlife_captioner,
    inputs=[gr.Image(label="Upload Wildlife Photo", type="pil")],
    outputs=[gr.Textbox(label="Generated Wildlife Caption")],
    title="Wildlife Photography Captioning",
    description="Upload wildlife photos and get natural, descriptive captions generated by a BLIP model.",
    allow_flagging="never",
    examples=[
        "examples/wildlife_lion.jpg",
        "examples/wildlife_eagle.jpg",
        "examples/wildlife_elephant.jpg",
    ],
)

# Launch app
demo.launch(share=True, server_port=PORT)
demo.close()


FileNotFoundError: [Errno 2] No such file or directory: 'examples/wildlife_lion.jpg'

IMPORTANT: You are using gradio version 3.37.0, however version 4.44.1 is available, please upgrade.
--------
